In [13]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords

tweet_id - int - index \
username - string\
text - string\
created_at - datetime\
location - string\
source - string\
url - string

text - split each sentence into individual words, without whitespace, puncutation, all lowercase.\
Each value of text is a list ['i', 'like', 'coffee']\
create word frequency dictionary ^

In [37]:
tw_mitaka = pd.read_csv("8_7_tw_mitaka.csv")
tw_mitaka['created_at'] = pd.to_datetime(tw_mitaka['created_at'])

In [38]:
tw_mitaka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    979 non-null    int64              
 1   username    979 non-null    object             
 2   text        979 non-null    object             
 3   created_at  979 non-null    datetime64[ns, UTC]
 4   location    675 non-null    object             
 5   source      979 non-null    object             
 6   url         979 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 53.7+ KB


In [39]:
tw_mitaka.head()

,tweet_id,username,text,created_at,location,source,url
0,1554430568087130115,Kejranaros,asa mitaka (war devil) edit // the perfect gir...,2022-08-02 11:35:02+00:00,NaN,Coub,https://twitter.com/Kejranaros/status/15544305...
1,1554428314210766849,ArcadaFox,Every week I go through the cycle of “oh I lov...,2022-08-02 11:26:04+00:00,"Georgia, USA",Twitter for iPhone,https://twitter.com/ArcadaFox/status/155442831...
2,1554417416427667457,ultramarynna,mitaka tuesday hooray https://t.co/uKvNTnZOA8,2022-08-02 10:42:46+00:00,Yanni · she/her 25↑,Twitter Web App,https://twitter.com/ultramarynna/status/155441...
3,1554406601821388800,rainyxink,is there a ship tag for mitaka and war I feel ...,2022-08-02 09:59:48+00:00,aroace - 23 - rey,Twitter for iPhone,https://twitter.com/rainyxink/status/155440660...
4,1554405676239278080,Sn1ke__,"Asa Mitaka, The war devil\nFirst Fan art \n#ch...",2022-08-02 09:56:07+00:00,In my house,Twitter for Android,https://twitter.com/Sn1ke__/status/15544056762...


In [40]:
#Stop words are omitted.
stops = set(stopwords.words('english'))

for i in range(len(tw_mitaka['text'])):
    tweet_text = []
    words = tw_mitaka['text'][i].lower().split()
    words = [''.join(letter for letter in word if letter not in (string.punctuation)) for word in words]
    for word in words:
        if word not in stops:
            tweet_text.append(word)
    tw_mitaka['text'][i] = tweet_text

#Need to remove "http..." at end of some tweets.
#Emotes and other languages other than English should be removed too. REGREX
#emote_dict

In [61]:
#Word frequency
word_freq = {}

for text_list in tw_mitaka['text']:
    for word in text_list:
        if word not in word_freq:
            word_freq[word] = 0
        word_freq[word] += 1

In [70]:
dict(list(word_freq.items())[0:5])

{'asa': 303, 'mitaka': 475, 'war': 545, 'devil': 539, 'edit': 6}

In [78]:
df_wordfreq = pd.DataFrame.from_dict(word_freq, orient='index', columns=['Freq'])

In [83]:
df_wordfreq.sort_values('Freq', ascending=False).head(10)

,Freq
war,545
devil,539
mitaka,475
asa,303
chainsawman,247
,205
yoru,158
man,139
chainsaw,136
like,111
